In [21]:
import lsst.afw.table as afwTable
import lsst.meas.algorithms.detection as detection
import astropy.table
import lsst.geom
from time import time
import astropy.units as u
import sys
from joblib import Parallel, delayed
import logging
import numpy as np

from salad.measure.fitting import logL_position
from salad.measure.forced import forced_exposures
from salad.images import Image
from salad.detection import DetectionCatalog
from salad.serialize import read
import logging


In [39]:
def detect(image : Image, threshold=3, no_masks=False, polarity="positive"):
    logging.basicConfig()
    log = logging.getLogger(__name__)

    if isinstance(image, Image):
        exposure = image.exposure
    else:
        exposure = image
        
    config = detection.SourceDetectionConfig()
    if no_masks:
        config.excludeMaskPlanes = []
    else:
        config.excludeMaskPlanes = [
            "EDGE",
            "SAT",
            "SUSPECT",
            "BAD",
            "NO_DATA",
            "STREAK",
            "CROSSTALK", # ? seems to cause noise
            "NOT_DEBLENDED" # ? seems to cause noise
        ]
    config.thresholdValue = threshold
    config.reEstimateBackground = False
    task = detection.SourceDetectionTask(config=config)

    exposure_date = exposure.visitInfo.getDate()
    half = (exposure.visitInfo.getExposureTime() / 2 + 1/2) / (24*60*60)
    exposure_date_jd = astropy.time.Time(exposure_date.get(exposure_date.JD) + half, format='jd', scale='utc')
    wcs = exposure.wcs
    point_to_sphere = wcs.getTransform().applyForward

    t2 = time()
    table = afwTable.SourceTable.makeMinimalSchema()
    results = task.run(table, exposure)
    t3 = time()

    footprints = getattr(results, polarity).getFootprints()
    peaks = []
    for footprint in footprints:
        peaks.append(footprint.peaks.asAstropy())
    peaks = astropy.table.vstack(peaks)
    t4 = time()
    coords = [point_to_sphere(lsst.geom.Point2D(_x, _y)) for _x, _y in zip(peaks['i_x'], peaks['i_y'])]
    ra = [float(c.getRa()) for c in coords]
    dec = [float(c.getDec()) for c in coords]
    peaks['ra'] = ra * u.radian
    peaks['dec'] = dec * u.radian
    t5 = time()

#     cutouts = [
#         exposure.getCutout(
#             lsst.geom.Point2D(
#                 row['i_x'], row['i_y']
#             ),
#             lsst.geom.Extent2I(
#                 30, 
#                 30
#             )
#         )
#         for row in peaks
#     ]
    forced = forced_exposures([exposure for i in range(len(peaks))], peaks)
    peaks = astropy.table.join(peaks, forced, keys_left=['i_x', 'i_y'], keys_right=['forced_i_x', 'forced_i_y'])
    peaks.remove_columns(['forced_exposure', 'forced_time', 'forced_detector', 'forced_i_x', 'forced_i_y'])
    peaks.sort("id")
    
    t6 = time()

    masked = np.zeros(exposure.mask.array.shape)
    masked_pixel_summary = {
        "total": np.product(exposure.mask.array.shape),
    }
    for m in config.excludeMaskPlanes:
        mask_plane_dict = exposure.mask.getMaskPlaneDict()
        b = mask_plane_dict[m]
        bit_mask = (exposure.mask.array >> b) & 1
        masked_pixel_summary[m] = bit_mask.sum()
        masked += bit_mask

    masked_pixel_summary['masked'] = (masked > 0).sum()
    t7 = time()

    log.error("run: %s", t3 - t2)
    log.error("astropy: %s", t4 - t3)
    log.error("ra/dec: %s", t5 - t4)
    log.error("forced: %s", t6 - t5)
    log.error("masks: %s", t7 - t6)
    log.error("found %s detections", len(peaks))

    return DetectionCatalog(peaks, exposure_date_jd, exposure.visitInfo.getId(), exposure.detector.getId(), masked_pixel_summary)


In [40]:
images = read("../search/tno_search/DEEP/20190403/A0c/detector_8/images.pkl")
detect(images[0], threshold=1).catalog

ERROR:__main__:run: 6.039714336395264
ERROR:__main__:astropy: 0.4541435241699219
ERROR:__main__:ra/dec: 0.49071264266967773
ERROR:__main__:forced: 18.537925004959106
ERROR:__main__:masks: 0.2091999053955078
ERROR:__main__:found 21609 detections


id,f_x,f_y,i_x,i_y,peakValue,significance,ra,dec,forced_logL,forced_a,forced_c,forced_flux,forced_flux_ref,forced_sigma,forced_sigma_ref,forced_SNR,forced_mag,forced_sigma_mag_high,forced_sigma_mag_low,forced_mask,forced_zero_point
,pix,pix,pix,pix,ct,,rad,rad,,,,,,,,,,,,,
int64,float32,float32,int32,int32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64
1,207.0,48.0,207,48,3.617172,2.7524900436401367,3.7641296597465574,-0.1978176131476013,-838.8192173633049,3.125722364938788e-05,0.014329236495999187,458.4295987618786,378.1587671810977,178.86476649273968,147.54562046540786,2.5629955398761375,24.55581456631833,-0.5369752792410409,-0.35766857069962654,18604,31.208996192141427
2,277.0,47.0,277,47,1.8539994,1.4087880849838257,3.7641278066318007,-0.19790673719967977,-791.1036271280743,3.11651637741616e-05,0.006561283540745494,210.5326186729466,173.6684449341671,179.12874915715824,147.76337987536593,1.1753145135191907,25.400697711163463,-2.0658405705229548,-0.6684198957703842,160,31.208996192141427
8,407.0,48.0,407,48,2.4230838,1.8389520645141602,3.7641280629492972,-0.19807228489760573,-823.104857687308,3.109057530937613e-05,0.008807582877782983,283.2878706855849,233.68428265790106,179.34349163791487,147.94052103728842,1.5795826661919146,25.07842624196008,-1.0885674272820616,-0.53251772673282,160,31.208996192141427
9,533.0,47.0,533,47,1.5713706,1.186301827430725,3.764125742159557,-0.1982327484186976,-815.0044351011454,3.0773301428216654e-05,0.005706024963777361,185.42128075167759,152.9540918116333,180.2656399184188,148.70120153840648,1.0286002414857984,25.538597250810003,-3.8896923046003558,-0.737374627257843,160,31.208996192141427
12,586.0,46.0,586,46,1.9177959,1.4605180025100708,3.764124010180655,-0.19830024650728814,-842.2049276385582,3.131208608152789e-05,0.006721946396952611,214.67577661387833,177.086137647314,178.7080018137679,147.41630521635412,1.2012656088986577,25.379538585434027,-1.9396736807687276,-0.6575835117589541,160,31.208996192141427
13,717.0,46.0,717,46,2.8103137,2.133129596710205,3.7641229285863775,-0.19846709960527853,-851.0446863688601,3.109884503564713e-05,0.010982674975309593,353.15378956101665,291.31670827191857,179.31964476847762,147.92084974474562,1.969409375180165,24.839086514982476,-0.7695719638144425,-0.4458351859259295,160,31.208996192141427
16,899.0,46.0,899,46,1.44677,1.1004594564437866,3.764121407940729,-0.1986989220076684,-850.8246815801754,3.123171502353286e-05,0.00483686301845364,154.8702341452943,127.75252072642247,178.93779583414326,147.60586239958448,0.8654976072737753,25.73407630475856,nan,-0.8338220619129899,160,31.208996192141427
17,992.0,50.0,992,50,2.4778512,1.895037055015564,3.7641258170111778,-0.19881740064283854,-803.7533711229614,3.1566713003887034e-05,0.00967030425921402,306.34498618919383,252.7041068515085,177.9857853043647,146.8205485165874,1.72117669770835,24.99346925008293,-0.9444594303561225,-0.49732821396334614,160,31.208996192141427


In [18]:
logL_position(images[0].exposure, lsst.geom.Point2D(1783, 53), [0, 0])

{'logL': -697.2842127128,
 'a': 3.103541694245316e-05,
 'c': 0.0180343240046082,
 'flux': 581.0885040806123,
 'flux_ref': 479.34014932655725,
 'sigma': 179.50279194171554,
 'sigma_ref': 148.07192792432087,
 'SNR': 3.237211509608672,
 'mag': 24.298390482910094,
 'sigma_mag_high': -0.40116007718586033,
 'sigma_mag_low': -0.2922726686906074,
 'mask': 165,
 'zero_point': 31.208996192141427}